# USAFacts.org
* Website is affiliated with the CDC, but is a private organization.  You can see USAFacts all over the CDS website
* https://www.cdc.gov/covid-data-tracker/

Data used in this notebook was downloaded from here:
https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

* https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv
* https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv
* https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv

In [51]:
import pandas as pd, pickle

### Download Data

In [27]:
url="https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv"
data_confirmed = pd.read_csv(url)

url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv"
data_deaths = pd.read_csv(url)

url = "https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv"
data_county_pop = pd.read_csv(url)

In [22]:
data_confirmed.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,61,67,68,74,84,91,93,103,103,110
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,205,208,216,222,224,227,231,243,244,254
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,51,53,58,59,61,67,69,74,79,79
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,44,44,45,46,46,46,46,46,49,50


In [23]:
data_deaths.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,3,4,4,4,4,4,4,4,4,4
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,5,5,5,5,6,7,7,8,8,8
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1


In [24]:
data_county_pop.head()

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


### Move dates to rows

In [28]:
data_confirmed = data_confirmed.melt(id_vars=['countyFIPS','County Name','State','stateFIPS'], var_name='Date', value_name='Confirmed')
data_confirmed.head()

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0


In [29]:
data_deaths = data_deaths.melt(id_vars=['countyFIPS','County Name','State','stateFIPS'], var_name='Date', value_name='Deaths')
data_deaths.head()

,countyFIPS,County Name,State,stateFIPS,Date,Deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0


### Create USAFacts CDC Master Data Table

In [31]:
data_confirmed.shape

(370620, 6)

In [32]:
data_deaths.shape

(370620, 6)

In [33]:
lefton = ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Date']
righton = ['countyFIPS', 'County Name', 'State', 'stateFIPS', 'Date']

data = pd.merge(data_confirmed, data_deaths, how='left', left_on=lefton, right_on=righton)

In [41]:
lefton = ['countyFIPS', 'County Name', 'State']
righton = ['countyFIPS', 'County Name', 'State']

data = pd.merge(data, data_county_pop, how='left', left_on=lefton, right_on=righton)

In [42]:
data.tail()

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths,population
370615,56037,Sweetwater County,WY,56,5/16/20,16,0.0,42343.0
370616,56039,Teton County,WY,56,5/16/20,69,1.0,23464.0
370617,56041,Uinta County,WY,56,5/16/20,8,0.0,20226.0
370618,56043,Washakie County,WY,56,5/16/20,6,0.0,7805.0
370619,56045,Weston County,WY,56,5/16/20,0,0.0,6927.0


#### Fix date data type

In [43]:
data.dtypes

countyFIPS       int64
County Name     object
State           object
stateFIPS        int64
Date            object
Confirmed        int64
Deaths         float64
population     float64
dtype: object

In [44]:
data['Date'] = data['Date'].astype('datetime64')

In [45]:
data.dtypes

countyFIPS              int64
County Name            object
State                  object
stateFIPS               int64
Date           datetime64[ns]
Confirmed               int64
Deaths                float64
population            float64
dtype: object

#### Dump to a pickle

In [49]:
data.shape

(370620, 8)

In [ ]:
pickle.dump(data, open('usafacts_cdc.p','wb'))

#### Quick NY Check

In [54]:
data = pickle.load(open('usafacts_cdc.p','rb'))

In [55]:
data.head()

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths,population
0,0,Statewide Unallocated,AL,1,2020-01-22,0,0.0,0.0
1,1001,Autauga County,AL,1,2020-01-22,0,0.0,55869.0
2,1003,Baldwin County,AL,1,2020-01-22,0,0.0,223234.0
3,1005,Barbour County,AL,1,2020-01-22,0,0.0,24686.0
4,1007,Bibb County,AL,1,2020-01-22,0,0.0,22394.0


#### Data appears to have county level detail needed in NYC

In [58]:
data[data['County Name']=='Bronx County'].tail()

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths,population
356510,36005,Bronx County,NY,36,2020-05-12,41441,3887.0,1418207.0
359705,36005,Bronx County,NY,36,2020-05-13,41677,3915.0,1418207.0
362900,36005,Bronx County,NY,36,2020-05-14,41980,3945.0,1418207.0
366095,36005,Bronx County,NY,36,2020-05-15,42519,3970.0,1418207.0
369290,36005,Bronx County,NY,36,2020-05-16,42812,3970.0,1418207.0


In [62]:
data[data['County Name']=='Queens County'].tail()

,countyFIPS,County Name,State,stateFIPS,Date,Confirmed,Deaths,population
356548,36081,Queens County,NY,36,2020-05-12,57391,5851.0,2253858.0
359743,36081,Queens County,NY,36,2020-05-13,57748,5878.0,2253858.0
362938,36081,Queens County,NY,36,2020-05-14,58084,5896.0,2253858.0
366133,36081,Queens County,NY,36,2020-05-15,58516,5930.0,2253858.0
369328,36081,Queens County,NY,36,2020-05-16,58841,5930.0,2253858.0
